# PY OOP

In [6]:
import pandas as pd
from scipy.integrate import odeint
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
import random
from statistics import mean 
import itertools
import ga 


In [73]:
class Parameters:
    def __init__(self):
        pass
    def initializeParameters(self,minimo,massimo,n=9):
        self.minimo=minimo
        self.massimo=massimo
        return [random.uniform(self.minimo,self.massimo) for i in range(0,n)]
    


    
    
        

In [115]:
class A:
    def __init__(self):
        self.param = Parameters().initializeParameters(0,1) 
        self.k1=self.param[0]
        self.k2=self.param[1]
        self.k3=self.param[2]
        self.k4=self.param[3]
        self.k5=self.param[4]
        self.k6=self.param[5]
        self.k7=self.param[6]
        self.k8=self.param[7]
        self.k9=self.param[8]
    
    @staticmethod
    def deriv(y,t,k1,k2,k3,k4,k5,k6,k7,k8,k9):  
        '''set di equazioni differenziali'''
        Sani,Infetti,Asintomatici,Morti,Guariti,Immuni = y 
        dSanidt =  -k1*Sani*Infetti -k2*Sani*Infetti-k3*Sani*Asintomatici- k4*Sani*Asintomatici
        dInfettidt = k1*Sani*Infetti+k4*Sani-k6*Infetti-k7*Infetti
        dAsintomaticidt =k2*Sani*Infetti+k3*Sani*Asintomatici-k5*Asintomatici
        dMortidt=k6*Infetti
        dGuaritidt = k5*Asintomatici+k7*Infetti-k8*Guariti-k9*Guariti
        dImmunidt = k9*Guariti
        return dSanidt,dInfettidt,dAsintomaticidt,dMortidt,dGuaritidt,dImmunidt  

    def perturbazione(self,ls):
        self.ls=ls
        sup = 0.2
        for i in range(len(self.ls)):
            if self.ls[i] < 0.5:
                self.ls[i]=random.uniform(self.ls[i],self.ls[i]+sup)
            else:
                self.ls[i]=random.uniform(0,self.ls[i]-sup)
        return(self.ls)

    def diffusione(self):
        orizzonte_temporale_in_gg=63
        Sani = 6036000
        Infetti=221
        Guariti=1
        Morti= 7
        Immuni = 8
        Asintomatici = 10
        y0 =  [Sani,Infetti,Morti,Guariti,Immuni,Asintomatici]
        t = np.linspace(0,orizzonte_temporale_in_gg,orizzonte_temporale_in_gg)
        ret = odeint(A.deriv, y0, t,args=(self.k1,self.k2,self.k3,self.k4,self.k5,self.k6,self.k7,self.k8,self.k9))
        return(ret)
   

    

In [116]:
a=A()
a.diffusione()

array([[ 6.03600000e+06,  2.21000000e+02,  7.00000000e+00,
         1.00000000e+00,  8.00000000e+00,  1.00000000e+01],
       [-1.20331883e-24,  7.27973186e+02,  3.26890975e+06,
         1.95745031e+02,  1.15745197e+06,  1.84136325e+05],
       [ 4.58167078e-24,  3.72277127e+02,  2.21164988e+06,
         2.95335552e+02,  1.42358694e+06,  5.43279648e+05],
       [ 9.61499933e-26,  1.90378254e+02,  1.49633841e+06,
         3.46264998e+02,  1.31758228e+06,  9.16067524e+05],
       [-8.21611298e-25,  9.73572561e+01,  1.01237934e+06,
         3.72309733e+02,  1.08756587e+06,  1.24011319e+06],
       [ 7.56456306e-23,  4.97873844e+01,  6.84946608e+05,
         3.85628712e+02,  8.44347474e+05,  1.49910076e+06],
       [-1.41899361e-23,  2.54606975e+01,  4.63415085e+05,
         3.92439885e+02,  6.31320037e+05,  1.69631959e+06],
       [ 1.76860084e-21,  1.30203088e+01,  3.13533257e+05,
         3.95923041e+02,  4.60368253e+05,  1.84192388e+06],
       [ 5.56872522e-22,  6.65843659e+00,  2.121

In [94]:
k1=0.2